In [1]:
import pandas as pd


# Chargement des fichiers CSV
benin_data = pd.read_csv("../data/dataBell_DNS Dataset/features_domain_benign_csv.csv")
malware_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_Malware.csv")
phishing_data = pd.read_csv("../data/Bell_DNS Dataset/features-domain_phishing.csv")
spam_data = pd.read_csv("../data/dataBell_DNS Dataset/features-domain_Spam.csv")

# Ajout de la colonne 'Class' 
benin_data['Class'] = 'Benin'
malware_data['Class'] = 'Malware'
phishing_data['Class'] = 'Phishing'
spam_data['Class'] = 'Spam'


# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)


In [2]:
# Spécifier la colonne cible je doit le faire avant l'encodage
y = all_data['Class']

In [7]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Encodage de la cible en labels numériques
label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)
y= label_encoder.fit_transform(y)


print(y)

[0 0 0 ... 3 3 3]


In [8]:
from sklearn.impute import SimpleImputer

# Concaténation de tous les ensembles de données
all_data = pd.concat([benin_data, malware_data, phishing_data, spam_data], ignore_index=True)

# Convertir les variables catégorielles en variables numériques
all_data_encoded = pd.get_dummies(all_data, columns=['Class'], drop_first=True)

# Exclure les colonnes non numériques
X = all_data_encoded.select_dtypes(include='number')

# Vérifier les colonnes avec des valeurs manquantes
missing_columns = X.columns[X.isnull().any()]

print("Missing value : {}", X[missing_columns])

# Imputation des valeurs manquantes
if not missing_columns.empty:
    imputer = SimpleImputer(strategy='mean')  # Vous pouvez également utiliser 'median' ou 'most_frequent'
    X[missing_columns] = imputer.fit_transform(X[missing_columns])



#apres remplissage avec la valuer moyenne    
print("Missing value : {}", X[missing_columns])

Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0        NaN
1            0.0         1.0        NaN
2            1.0         1.0        NaN
3            1.0         1.0        NaN
4            0.0         7.0        NaN
...          ...         ...        ...
50154        0.0        -1.0        NaN
50155        0.0        -1.0        NaN
50156        0.0        -1.0        NaN
50157        0.0        -1.0        NaN
50158        0.0        -1.0        NaN

[50159 rows x 3 columns]
Missing value : {}        subdomain  Alexa_Rank  Page_Rank
0            0.0         1.0       -1.0
1            0.0         1.0       -1.0
2            1.0         1.0       -1.0
3            1.0         1.0       -1.0
4            0.0         7.0       -1.0
...          ...         ...        ...
50154        0.0        -1.0       -1.0
50155        0.0        -1.0       -1.0
50156        0.0        -1.0       -1.0
50157        0.0        -1.0       -1.0
50158        0.0

In [4]:
num_features = X.shape[1]
print("Nombre total de caractéristiques numeriques (car les autres ont été ignorer):", num_features)


#donc dans notre fonction il faut passer un nombre de caracteristiques inferieur ou egale a 13

Nombre total de caractéristiques numeriques (car les autres ont été ignorer): 13


In [9]:
from sklearn.tree import DecisionTreeRegressor #Comme estimateur pour la sélection des caractéristiques.
from sklearn.model_selection import StratifiedKFold #Pour effectuer une validation croisée stratifiée.
from sklearn.feature_selection import RFECV #La classe qui effectue la sélection de caractéristiques



def rfe_cross_validation(X, y, feature_number):
    # Créer une instance de RFECV avec l'estimateur précédemment produit 
    rfe_cv = RFECV(estimator=DecisionTreeRegressor(), step=1, cv=StratifiedKFold(10),
                   verbose=1, min_features_to_select=feature_number, n_jobs=4)

    # Ajuster RFECV sur les données
    rfe_cv.fit(X, y)

    # Transformer X pour ne conserver que les caractéristiques sélectionnées
    X_selected = rfe_cv.transform(X)

    # Créer un DataFrame avec les noms des caractéristiques et leurs importances
    feature_set = pd.DataFrame()
    # feature_set['attr'] = X.columns
    feature_set['attr'] = X.columns[rfe_cv.support_]  # Utiliser seulement les caractéristiques sélectionnées pour ne pas avoir un probleme de hors page/marge comme la ligne precedente
    feature_set['importance'] = rfe_cv.estimator_.feature_importances_

    # Trier par ordre décroissant d'importance
    feature_set = feature_set.sort_values(by='importance', ascending=False)

    # Retourner les noms des caractéristiques les plus importantes
    return feature_set['attr'].head(feature_number).values

In [10]:
# Appel de la fonction et impression des features importants
important_features_rfe = rfe_cross_validation(X, y, feature_number=6)
print("Caractéristiques importantes (RFECV):")
print(important_features_rfe)

Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Caractéristiques importantes (RFECV):
['Alexa_Rank' 'entropy' 'len' 'numeric_percentage' 'subdomain'
 'puny_coded']
